# L4: Process Inputs: Chain of Thought Reasoning

## 설정 
### API 키와 관련 Python 라이브러리를 로드합니다. 
이 과정에서는 OpenAI API 키를 로드하는 몇 가지 코드를 제공했습니다.

In [1]:
import os
import openai
import sys
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

## Chain-of-Thought Prompting

In [3]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
Must reply in Korean.
"""

In [4]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking about the price difference between the BlueWave Chromebook and the TechPro Desktop.

Step 2:#### Both the BlueWave Chromebook and the TechPro Desktop are available products.

Step 3:#### The user assumes that the BlueWave Chromebook is more expensive than the TechPro Desktop.

Step 4:#### The assumption made by the user is incorrect. The TechPro Desktop is priced at $999.99, while the BlueWave Chromebook is priced at $249.99. Therefore, the BlueWave Chromebook is actually less expensive than the TechPro Desktop.

Response to user:#### 블루웨이브 크롬북은 테크프로 데스크탑보다 싸요. 테크프로 데스크탑은 $999.99이고, 블루웨이브 크롬북은 $249.99입니다.


In [5]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages, model='gpt-4')
print(response)

Step 1:#### The user is asking a question about specific products, namely the BlueWave Chromebook and the TechPro Desktop.

Step 2:#### Both the BlueWave Chromebook and the TechPro Desktop are in the list of available products.

Step 3:#### The user is making an assumption that the BlueWave Chromebook is more expensive than the TechPro Desktop.

Step 4:#### According to the product information, the assumption is incorrect. The TechPro Desktop is priced at $999.99, while the BlueWave Chromebook is priced at $249.99. Therefore, the TechPro Desktop is more expensive than the BlueWave Chromebook.

Response to user:#### 사용자님, 제품 가격에 대해 잘못 이해하신 것 같습니다. TechPro Desktop의 가격은 $999.99이고, BlueWave Chromebook의 가격은 $249.99입니다. 따라서, TechPro Desktop이 BlueWave Chromebook보다 비싸습니다.


In [6]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### 사용자가 특정 제품에 대해 질문하는지 여부를 판단합니다. 제품 카테고리는 고려하지 않습니다.

Step 2:#### 사용자가 특정 제품에 대해 묻는 경우, 다음 목록에 해당 제품이 있는지 확인합니다.
모든 판매 제품:
1. 제품: TechPro 울트라북
   카테고리: 컴퓨터 및 노트북
   브랜드: TechPro
   모델 번호: TP-UB100
   보증: 1년
   평점: 4.5
   특징: 13.3인치 디스플레이, 8GB RAM, 256GB SSD, 인텔 코어 i5 프로세서
   설명: 일상적인 사용을 위한 슬림하고 가벼운 울트라북
   가격: $799.99

2. 제품: BlueWave 게이밍 노트북
   카테고리: 컴퓨터 및 노트북
   브랜드: BlueWave
   모델 번호: BW-GL200
   보증: 2년
   평점: 4.7
   특징: 15.6인치 디스플레이, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   설명: 몰입형 경험을 위한 고성능 게이밍 노트북
   가격: $1199.99

3. 제품: PowerLite 컨버터블
   카테고리: 컴퓨터 및 노트북
   브랜드: PowerLite
   모델 번


In [7]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages, model='gpt-4')
print(response)

Step 1:#### The user is asking about a product category (TVs), not a specific product.

Step 2:#### The user's query does not mention any of the specific products in the list.

Step 3:#### The user is not making any assumptions about specific products.

Step 4:#### Not applicable as the user did not make any assumptions about specific products.

Response to user:#### 죄송합니다, 저희는 컴퓨터와 노트북만 판매하고 있습니다. TV는 판매하고 있지 않습니다. 다른 질문이 있으시면 언제든지 물어보세요.


## Inner Monologue
- LLM에 구분 기호로 추론 단계를 구분하도록 요청했기 때문에 사용자에게 표시되는 최종 출력에서 사고 연쇄 추론을 숨길 수 있습니다.

In [8]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."
    
print(final_response)

죄송합니다, 저희는 컴퓨터와 노트북만 판매하고 있습니다. TV는 판매하고 있지 않습니다. 다른 질문이 있으시면 언제든지 물어보세요.
